# S&P500 Dataset Cleaning



Link to dataset: https://www.kaggle.com/datasets/awadhi123/finance-data-sp-500

Description: the dataset contains the candle stick data necessary for the S&P500 index from 2 Jan 2000 to 10 Jul 2020

__Note:__ 
* I kept the variable "volume" since I think it might be a relevant feature
* 'up/down' column: 1 represents the day's closing price is higher than its opening price; -1, lower; 0, otherwise. 

In [1]:
import pandas as pd
df = pd.read_csv("s&p500.csv")   #import the data and read as df

In [2]:
df.head()

,Date,Open,High,Low,Close*,Adj Close**,Volume
0,1980-01-02,0,108.43,105.29,105.76,105.76,"40,610,000"
1,1980-01-03,0,106.08,103.26,105.22,105.22,"50,480,000"
2,1980-01-04,0,107.08,105.09,106.52,106.52,"39,130,000"
3,1980-01-07,0,107.8,105.8,106.81,106.81,"44,500,000"
4,1980-01-08,0,109.29,106.29,108.95,108.95,"53,390,000"


Adjusted close price is not relevant, I dropped it: 

In [3]:
df = df.drop(['Adj Close**'], axis = 1)

#rename close*
df.rename(columns = {'Close*':'Close'}, inplace = True)
df.head()

,Date,Open,High,Low,Close,Volume
0,1980-01-02,0,108.43,105.29,105.76,"40,610,000"
1,1980-01-03,0,106.08,103.26,105.22,"50,480,000"
2,1980-01-04,0,107.08,105.09,106.52,"39,130,000"
3,1980-01-07,0,107.8,105.8,106.81,"44,500,000"
4,1980-01-08,0,109.29,106.29,108.95,"53,390,000"


In [4]:
# print("The first workday of 2018 is " + df["Date"][4528])
# print("The last workday of 2018 is " + df["Date"][4778])

I dropped all observations outside the above range and reset the indices

In [5]:
#drop unnecessary observations since we only look at 2018's data
# df = df[4528:4779].reset_index()

In [6]:
#check the range of dates
# df["Date"]

In [7]:
# new column 'up/down': if closing price is higher than opening 
# price, displays 1; if lower. displays -1; else displays 0
df['up/down'] = 0
for i in range(df.shape[0]): 
    if df['Open'][i] > df['Close'][i]: 
        df['up/down'][i] = -1
    elif df['Open'][i] < df['Close'][i]:
        df['up/down'][i] = 1

<ipython-input-7-a432cb372136>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['up/down'][i] = 1
<ipython-input-7-a432cb372136>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['up/down'][i] = -1


In [8]:
# new column for the true values, if next day's opening price
# is higher than today's price, display 1; if lower display -1
# 1,0 = buy, -1 = sell
df['true_value'] = 0
for i in range(df.shape[0] - 1): 
    if df['Close'][i] < df['Close'][i+1]: 
        df['true_value'][i] = 1
    elif df['Close'][i] > df['Close'][i+1]: 
        df['true_value'][i] = -1
    else:
        df['true_value'][i] = 1
        
#last day buy
df['true_value'][-1:] = 1
        

<ipython-input-8-bd132e241cb0>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['true_value'][i] = -1
<ipython-input-8-bd132e241cb0>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['true_value'][i] = 1
<ipython-input-8-bd132e241cb0>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['true_value'][i] = 1
<ipython-input-8-bd132e241cb0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

In [9]:
# df['Close'][3]

In [10]:
df.head()

,Date,Open,High,Low,Close,Volume,up/down,true_value
0,1980-01-02,0,108.43,105.29,105.76,"40,610,000",1,-1
1,1980-01-03,0,106.08,103.26,105.22,"50,480,000",1,1
2,1980-01-04,0,107.08,105.09,106.52,"39,130,000",1,1
3,1980-01-07,0,107.8,105.8,106.81,"44,500,000",1,1
4,1980-01-08,0,109.29,106.29,108.95,"53,390,000",1,1


In [11]:
#str date to datetime object
from datetime import datetime
df['Date']=df['Date'].apply(lambda row: datetime.strptime(row, "%Y-%m-%d"))

In [12]:
df.dtypes

Date          datetime64[ns]
Open                  object
High                  object
Low                   object
Close                 object
Volume                object
up/down                int64
true_value             int64
dtype: object

In [13]:
#convert data types from str to floats & remove ,
df['Open'] = df['Open'].str.replace(',', '').astype(float)
df['High'] = df['High'].str.replace(',', '').astype(float)
df['Low'] = df['Low'].str.replace(',', '').astype(float)
df['Close'] = df['Close'].str.replace(',', '').astype(float)
df['Volume'] = df['Volume'].str.replace(',', '').astype(float)

In [14]:
df.head()

,Date,Open,High,Low,Close,Volume,up/down,true_value
0,1980-01-02,0.0,108.43,105.29,105.76,40610000.0,1,-1
1,1980-01-03,0.0,106.08,103.26,105.22,50480000.0,1,1
2,1980-01-04,0.0,107.08,105.09,106.52,39130000.0,1,1
3,1980-01-07,0.0,107.80,105.80,106.81,44500000.0,1,1
4,1980-01-08,0.0,109.29,106.29,108.95,53390000.0,1,1


In [15]:
from sklearn.preprocessing import StandardScaler

#normalize data using sklearn
normalizer = StandardScaler()
normalizer
df_dropped = df.drop('Date', axis = 1)
df_dropped = df_dropped.drop('up/down', axis = 1)
df_dropped = df_dropped.drop('true_value', axis = 1)
normalized_df = pd.DataFrame(normalizer.fit_transform(df_dropped), columns = df_dropped.columns)
normalized_df.insert(loc = 0, column = 'Date', value = df['Date'])
normalized_df.insert(loc = 6, column = 'up/down', value = df['up/down'])
normalized_df.insert(loc = 7, column = 'true_value', value = df['true_value'])

In [16]:
normalized_df

,Date,Open,High,Low,Close,Volume,up/down,true_value
0,1980-01-02,-1.157349,-1.064447,-1.065108,-1.066003,-0.960949,1,-1
1,1980-01-03,-1.157349,-1.066807,-1.067169,-1.066548,-0.955609,1,1
2,1980-01-04,-1.157349,-1.065803,-1.065311,-1.065236,-0.961750,1,1
3,1980-01-07,-1.157349,-1.065080,-1.064590,-1.064943,-0.958844,1,1
4,1980-01-08,-1.157349,-1.063584,-1.064093,-1.062783,-0.954035,1,1
...,...,...,...,...,...,...,...,...
10693,2022-05-27,2.928087,3.002164,2.967795,3.024203,0.943378,1,-1
10694,2022-05-31,3.001891,3.012054,2.995665,2.997870,1.826122,-1,-1
10695,2022-06-01,3.000579,3.010247,2.964160,2.966662,1.259950,-1,1
10696,2022-06-02,2.946102,3.021262,2.964688,3.042956,0.967383,1,-1


In [17]:
normalized_df['true_value'].value_counts()

 1    5728
-1    4970
Name: true_value, dtype: int64

In [19]:
# save file as csv
normalized_df.to_csv('s&p500_processed.csv')